In [205]:
import joblib
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import os
import random
import pickle

from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import simpleaudio as sa

def get_classifier(y_test,pred):
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test,pred)
    
    print("accuracy:{},Precision:{}, Recall:{}".format(accuracy,precision,recall))

X_train1=[]
y_train1=[]
X_test1=[]
y_test1=[]

cry=0
noise=1

In [206]:
def butter_bandpass(lowcut, highcut,fs, order=5):
    nyq= 0.5*fs
    low= lowcut/nyq
    high= highcut/nyq
    b,a = butter(order, [low,high], btype='band')
    return b,a

def butter_bandpass_filter(data, lowcut,highcut, fs, order=5):
    b,a= butter_bandpass( lowcut, highcut, fs, order)
    y=lfilter(b,a,data)
    return y

def playSineWaveSound():
    frequency = 400  # Our played note will be 440 Hz
    fs = 44100  # 44100 samples per second
    seconds = 3  # Note duration of 3 seconds
    t = np.linspace(0, seconds, seconds * fs, False)
    note = np.sin(frequency * t * 2 * np.pi)
    audio = note * (2 ** 15 - 1) / np.max(np.abs(note))
    audio = audio.astype(np.int16)
    play_obj = sa.play_buffer(audio, 1, 2, fs)
    play_obj.wait_done()
    plt.title('Sine signal')
    plt.plot(range(0,audio.shape[0]), audio, label='Sine signal')
    plt.show()

def callBandPassSample(a):
    audio_wave_data, sr = librosa.load(a) # returns numpy.ndarray
    audio = audio_wave_data * (2 ** 15 - 1) / np.max(np.abs(audio_wave_data))
    YY = []
    for i in range(1, 4001, 200):
        y0= butter_bandpass_filter(audio, i,i+200, sr, 3)
        YY.append(y0)
    
    '''
    y1= butter_bandpass_filter(audio, 200,400, sr, 3) # 250, 450
    y2= butter_bandpass_filter(audio, 400,600, sr, 3)
    y3= butter_bandpass_filter(audio, 600,800, sr, 3)
    y4= butter_bandpass_filter(audio, 800,1000, sr, 3)
    y5= butter_bandpass_filter(audio, 1000,1200, sr, 3)
    xx= range(0,audio.shape[0] )
    '''
    '''
    plt.title('Original signal')
    plt.plot(xx, audio, label='')
    plt.show()

    plt.title('Bandpass signal')
    plt.plot(xx, y1, label='')
    plt.show()
    '''
    #print(len(xx))
    #print(len(y1))
    '''
    audio2 = y1.astype(np.int16)
    play_obj = sa.play_buffer(audio2, 1, 2, sr)
    play_obj.wait_done()
    '''
    return YY

if __name__ == "__main__":
    a = "./train/1.wav"
    callBandPassSample(a)
    pass

In [207]:
# train 데이터
c = 0
for filename in os.listdir("./train/"):
    print(c)
    c+=1
    #if c == 4:
    #    break
    try:
        if '.wav' not in filename:
            continue
         
        y,sr = librosa.load("./train/"+filename)
        #print(len(y))
        #print(filename)
        y = callBandPassSample("./train/"+filename)
        '''
        mfcc = librosa.feature.mfcc(y=y,sr=sr)
        #print(np.shape(mfcc))
        mfcc_avg = []
        sum_ = 0 
        
        for i in mfcc:
            for j in i:
                sum_ += j
            mfcc_avg.append(sum_/10261) 
        '''
        
        X_train1.append(y)
        
        if '_no_cry.wav' in filename:
            y_train1.append(noise)
        else:
            y_train1.append(cry)
    
    
    except Exception as e:
        print(filename,e)
        raise

# test 데이터
for filename in os.listdir("./test/"):
    print(c)
    c+=1
    try:
        if '.wav' not in filename:
            continue
         
        y,sr = librosa.load("./test/"+filename)
        y = callBandPassSample("./test/"+filename)
        '''
        mfcc = librosa.feature.mfcc(y=y,sr=sr)
        mfcc_avg = []
        sum_ = 0 
        for i in mfcc:
            for j in i:
                sum_ += j
            mfcc_avg.append(sum_/10261)
        '''
        X_test1.append(y)
        
        if '_no_cry.wav' in filename:
            y_test1.append(noise)
        else:
            y_test1.append(cry)
        
    
    except Exception as e:
        print(filename,e)
        raise

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


In [208]:
# 78 + 28
x = X_train1 + X_test1
y = y_train1 + y_test1

In [209]:
len(x), len(y)

(106, 106)

In [210]:
# x[24:], x[:24]
# y[24:], y[:24] 81 21
X_train = x[24:]
y_train = y[24:]
X_test = x[:24]
y_test = y[:24]

In [211]:
len(X_train), len(y_train), len(X_test), len(y_test)

(82, 82, 24, 24)

In [247]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = X_train[i][j][:40000]

for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = X_test[i][j][:40000]

In [249]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        print(i, j, end=", ")
        print(np.shape(X_train[i][j]))
        
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        print(i, j, end=", ")
        print(np.shape(X_test[i][j]))

0 0, (40000,)
0 1, (40000,)
0 2, (40000,)
0 3, (40000,)
0 4, (40000,)
0 5, (40000,)
0 6, (40000,)
0 7, (40000,)
0 8, (40000,)
0 9, (40000,)
0 10, (40000,)
0 11, (40000,)
0 12, (40000,)
0 13, (40000,)
0 14, (40000,)
0 15, (40000,)
0 16, (40000,)
0 17, (40000,)
0 18, (40000,)
0 19, (40000,)
1 0, (40000,)
1 1, (40000,)
1 2, (40000,)
1 3, (40000,)
1 4, (40000,)
1 5, (40000,)
1 6, (40000,)
1 7, (40000,)
1 8, (40000,)
1 9, (40000,)
1 10, (40000,)
1 11, (40000,)
1 12, (40000,)
1 13, (40000,)
1 14, (40000,)
1 15, (40000,)
1 16, (40000,)
1 17, (40000,)
1 18, (40000,)
1 19, (40000,)
2 0, (40000,)
2 1, (40000,)
2 2, (40000,)
2 3, (40000,)
2 4, (40000,)
2 5, (40000,)
2 6, (40000,)
2 7, (40000,)
2 8, (40000,)
2 9, (40000,)
2 10, (40000,)
2 11, (40000,)
2 12, (40000,)
2 13, (40000,)
2 14, (40000,)
2 15, (40000,)
2 16, (40000,)
2 17, (40000,)
2 18, (40000,)
2 19, (40000,)
3 0, (40000,)
3 1, (40000,)
3 2, (40000,)
3 3, (40000,)
3 4, (40000,)
3 5, (40000,)
3 6, (40000,)
3 7, (40000,)
3 8, (40000,)
3 9,

38 16, (40000,)
38 17, (40000,)
38 18, (40000,)
38 19, (40000,)
39 0, (40000,)
39 1, (40000,)
39 2, (40000,)
39 3, (40000,)
39 4, (40000,)
39 5, (40000,)
39 6, (40000,)
39 7, (40000,)
39 8, (40000,)
39 9, (40000,)
39 10, (40000,)
39 11, (40000,)
39 12, (40000,)
39 13, (40000,)
39 14, (40000,)
39 15, (40000,)
39 16, (40000,)
39 17, (40000,)
39 18, (40000,)
39 19, (40000,)
40 0, (40000,)
40 1, (40000,)
40 2, (40000,)
40 3, (40000,)
40 4, (40000,)
40 5, (40000,)
40 6, (40000,)
40 7, (40000,)
40 8, (40000,)
40 9, (40000,)
40 10, (40000,)
40 11, (40000,)
40 12, (40000,)
40 13, (40000,)
40 14, (40000,)
40 15, (40000,)
40 16, (40000,)
40 17, (40000,)
40 18, (40000,)
40 19, (40000,)
41 0, (40000,)
41 1, (40000,)
41 2, (40000,)
41 3, (40000,)
41 4, (40000,)
41 5, (40000,)
41 6, (40000,)
41 7, (40000,)
41 8, (40000,)
41 9, (40000,)
41 10, (40000,)
41 11, (40000,)
41 12, (40000,)
41 13, (40000,)
41 14, (40000,)
41 15, (40000,)
41 16, (40000,)
41 17, (40000,)
41 18, (40000,)
41 19, (40000,)
42 0, 

77 15, (40000,)
77 16, (40000,)
77 17, (40000,)
77 18, (40000,)
77 19, (40000,)
78 0, (40000,)
78 1, (40000,)
78 2, (40000,)
78 3, (40000,)
78 4, (40000,)
78 5, (40000,)
78 6, (40000,)
78 7, (40000,)
78 8, (40000,)
78 9, (40000,)
78 10, (40000,)
78 11, (40000,)
78 12, (40000,)
78 13, (40000,)
78 14, (40000,)
78 15, (40000,)
78 16, (40000,)
78 17, (40000,)
78 18, (40000,)
78 19, (40000,)
79 0, (40000,)
79 1, (40000,)
79 2, (40000,)
79 3, (40000,)
79 4, (40000,)
79 5, (40000,)
79 6, (40000,)
79 7, (40000,)
79 8, (40000,)
79 9, (40000,)
79 10, (40000,)
79 11, (40000,)
79 12, (40000,)
79 13, (40000,)
79 14, (40000,)
79 15, (40000,)
79 16, (40000,)
79 17, (40000,)
79 18, (40000,)
79 19, (40000,)
80 0, (40000,)
80 1, (40000,)
80 2, (40000,)
80 3, (40000,)
80 4, (40000,)
80 5, (40000,)
80 6, (40000,)
80 7, (40000,)
80 8, (40000,)
80 9, (40000,)
80 10, (40000,)
80 11, (40000,)
80 12, (40000,)
80 13, (40000,)
80 14, (40000,)
80 15, (40000,)
80 16, (40000,)
80 17, (40000,)
80 18, (40000,)
80 19,

In [250]:
len(X_train[0][0])

40000

In [285]:
np.shape(y_train)

(82,)

In [252]:
X_train2=np.array(X_train)
y_train2=np.array(y_train)
X_test2=np.array(X_test)
y_test2=np.array(y_test)

X_train2 = X_train2.reshape(X_train2.shape[0], X_train2.shape[1], X_train2.shape[2])
X_test2 = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], X_train2.shape[2])

In [253]:
len(X_train[1][1])

40000

In [256]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM

'''
# 1. 데이터
x = array([[1,2,3], [2,3,4], [3,4,5], [4,5,6]])
y = array([4,5,6,7])
 
print('x shape : ', x.shape) # (4,3)
print('y shape : ', y.shape) # (4,)
#  x  y
# 123 4
# 234 5
# 345 6
# 456 7
 
print(x)
print('-------x reshape-----------')
x = x.reshape((x.shape[0], x.shape[1], 1)) # (4,3,1) reshape 전체 곱 수 같아야 4*3=4*3*1
print('x shape : ', x.shape)
print(x)
#  x        y
# [1][2][3] 4
# .....
 '''

model = Sequential()
# 2. 모델 구성
model.add(LSTM(128, activation = 'relu', input_shape=(20,40000)))
#model.add(LSTM(10))
# DENSE와 사용법 동일하나 input_shape=(열, 몇개씩잘라작업)
model.add(Dense(5))
model.add(Dense(1))
model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               20546048  
_________________________________________________________________
dense_22 (Dense)             (None, 5)                 645       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 6         
Total params: 20,546,699
Trainable params: 20,546,699
Non-trainable params: 0
_________________________________________________________________


In [402]:
from sklearn import ensemble
from sklearn import linear_model
from tensorflow import keras
# X_train1, y_train1
import kerastuner as kt
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(128, activation='relu', input_shape=(20, 40000)))
    #model.add(keras.layers.LSTM(64, activation='relu'))
    #model.add(keras.layers.LSTM(32, activation='relu'))
    #model.add(keras.layers.Flatten(input_shape=(20, 40000)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    model.add(keras.layers.Dense(1))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss=keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model


from kerastuner.tuners import Hyperband


tuner = kt.Hyperband(
        build_model, # HyperModel
        objective ='val_accuracy', #  최적화할 하이퍼모델
        max_epochs =100, # 각 모델별 학습 회수
        factor = 3,    # 한 번에 훈련할 모델 수 결정 변수
        directory ='./', # 사용된 parameter 저장할 폴더
        project_name ='HB_LSTM') # 사용된 parameter 저장할 폴더

INFO:tensorflow:Reloading Oracle from existing project ./HB_LSTM\oracle.json
INFO:tensorflow:Reloading Tuner from ./HB_LSTM\tuner0.json


In [403]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [404]:
tuner.search(X_train2, y_train2, epochs=100, validation_data=(X_test2, y_test2))
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 46 Complete [00h 00m 14s]
val_accuracy: 0.5833333134651184

Best val_accuracy So Far: 0.75
Total elapsed time: 00h 02m 24s
INFO:tensorflow:Oracle triggered exit


In [406]:
print(f'''
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
''')


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer is 0.0001



In [407]:
models = tuner.get_best_models(num_models=2)
tuner.results_summary()

Results summary
Results in ./HB_LSTM
Showing 10 best trials
Trial summary
Hyperparameters:
units: 448
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.75
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.7083333134651184
Trial summary
Hyperparameters:
units: 256
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.7083333134651184
Trial summary
Hyperparameters:
units: 64
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.625
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.625
Trial summary
Hyperparameters:
units: 256
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.625
Trial summary
Hyperparameters:


In [415]:
model = tuner.hypermodel.build(best_hps)
model.fit(X_train2, y_train2, epochs = 2, validation_data = (X_test2, y_test2))

Epoch 1/2
3/3 [==============================] - 7s 2s/step - loss: 222.1673 - accuracy: 0.5976 - val_loss: 93.3529 - val_accuracy: 0.5417
Epoch 2/2
3/3 [==============================] - 4s 1s/step - loss: 140.5214 - accuracy: 0.5488 - val_loss: 98.2513 - val_accuracy: 0.6250


In [416]:
yhat = model.predict(X_test2)
print(yhat)

[[ -792.46796 ]
 [ -675.0257  ]
 [   54.782562]
 [ -200.95335 ]
 [  140.99066 ]
 [ -535.38367 ]
 [  -32.279137]
 [ -213.80956 ]
 [  -16.247322]
 [ -410.73782 ]
 [ -318.26993 ]
 [ -517.4133  ]
 [ -838.83264 ]
 [ -242.10913 ]
 [  -58.525497]
 [ -970.51807 ]
 [ -247.08377 ]
 [ -400.54803 ]
 [   17.92302 ]
 [   82.86371 ]
 [  -24.338219]
 [ -739.5912  ]
 [-1373.5304  ]
 [ -199.33598 ]]


In [430]:
cor = 0
y_list = []
for i,j in zip(yhat, y_test2):
    print(i[0], end=" \t ")
    if i[0] < -100:
        y_list.append(0)
        print(0, j)
        if 0 == j:
            cor += 1
    else:
        y_list.append(1)
        print(1, j)
        if 1 == j:
            cor += 1
print(cor/len(y_test))

-792.46796 	 0 0
-675.0257 	 0 0
54.782562 	 1 1
-200.95335 	 0 0
140.99066 	 1 1
-535.38367 	 0 0
-32.279137 	 1 1
-213.80956 	 0 0
-16.247322 	 1 1
-410.73782 	 0 0
-318.26993 	 0 1
-517.4133 	 0 0
-838.83264 	 0 1
-242.10913 	 0 0
-58.525497 	 1 1
-970.51807 	 0 0
-247.08377 	 0 1
-400.54803 	 0 0
17.92302 	 1 1
82.86371 	 1 0
-24.338219 	 1 1
-739.5912 	 0 1
-1373.5304 	 0 0
-199.33598 	 0 0
0.7916666666666666


In [418]:
from sklearn.metrics import classification_report

y_true = y_test
y_pred = y_list

print(classification_report(y_true, y_pred, target_names=['class 0', 'class 1']))

              precision    recall  f1-score   support

     class 0       0.75      0.92      0.83        13
     class 1       0.88      0.64      0.74        11

    accuracy                           0.79        24
   macro avg       0.81      0.78      0.78        24
weighted avg       0.81      0.79      0.79        24



In [423]:
import tensorflow as tf

model.save('BPFLSTM_Crying.h5')
new_model = tf.keras.models.load_model('BPFLSTM_Crying.h5')

#test = np.array(X_test1[0]).reshape((X_test1[0].shape[0], X_test1[0].shape[1], 1))

new_hat = new_model.predict(np.array([X_test2[0]]))
new_hat[0][0]
#test_loss, test_acc = new_model.evaluate(X_test,  y_test, verbose=2)

-792.4557

In [434]:
import pyaudio
import wave
import numpy as np
import os
import librosa
import joblib
import tkinter
import tkinter.font
import time
import tensorflow as tf

from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import simpleaudio as sa


FORMAT = pyaudio.paInt16

CHANNELS = 1

RATE = 16000

CHUNK = 1024
 
RECORD_SECONDS = 5

limit_db = 1500

WAVE_OUTPUT_FILENAME = "./11.wav"

cry_sw=0 
Sound_1 = 0;    # 사운드 모듈 1
Sound_2 = 0;    # 사운드 모듈 2
count = 0;       # 사운드 모듈 데이터 갯수
result = 0;     # 데이터의 연산 값
stack = 3;       # 울음소리 감지
S_stack = 0;     # stack 이전값 저장
noise = 0;      # stack 초기화

audio = pyaudio.PyAudio()

# start Recording

stream = audio.open(format=pyaudio.paInt16, 

                channels=CHANNELS, 

                rate=RATE, 

                input=True, 

                input_device_index=1,

                frames_per_buffer=CHUNK)

loud = []
sum__ = 0
fre_list = []

try:
    while True:

        data = np.fromstring(stream.read(CHUNK),dtype=np.int16)
        #print(int(np.average(np.abs(data))))

        if(count >= 10):
            count = 0
            result = Sound_1
            result = result // 10
            if(result >= limit_db):
                stack = stack +1
            print(result)

            if(stack == S_stack):
                noise = noise + 1
            if(stack > S_stack):
                noise = 0      # stack에 새로운 값 입력시 noise 초기화
            
            S_stack = stack     #S_stack에 이전 stack값 저장
            if(noise >=10):
                noise=0
                stack=0

            if(stack > 3): #stack에 따라 울음감지
                print("Crying Detection")
                stack = stack - 2
                cry_sw = 1



    
            Sound_2 = Sound_1
            Sound_1 = 0
            count = 0


        count = count +1
        Sound_add = int(np.average(np.abs(data)))

        Sound_1 = abs(Sound_add) + Sound_1


        if cry_sw == 1:
            cry_sw = 0
            print ("recording...")
            frames = []
            
            for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):

                data = stream.read(CHUNK)

                frames.append(data)

            print ("finished recording")


            sum__ = 0

            waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
            waveFile.setnchannels(CHANNELS)
            waveFile.setsampwidth(audio.get_sample_size(FORMAT))
            waveFile.setframerate(RATE)
            waveFile.writeframes(b''.join(frames))
            waveFile.close()  

            X_test = []
            y , sr = librosa.load(WAVE_OUTPUT_FILENAME) # file_name librosa.load() : 오디오 파일을 로드한다. 'crysound.wav'
            #y = callBandPassSample(WAVE_OUTPUT_FILENAME)
            y = callBandPassSample(WAVE_OUTPUT_FILENAME)
            X_test.append(y)

            X_test11 = X_test
            for i in range(len(X_test11)):
                for j in range(len(X_test11[i])):
                    X_test11[i][j] = X_test11[i][j][:40000]
                    
            X_test11 = np.array(X_test11)
            X_test11 = X_test11.reshape((X_test11.shape[0], X_test11.shape[1], X_test11.shape[2])) # LSTM
            
            for i in range(len(X_test11)):
                for j in range(len(X_test11[i])):
                    X_test11[i][j] = X_test11[i][j][:40000]

            #X_test22 = X_test1.reshape((X_test1.shape[0], X_test1.shape[1], 1, 1)) # CNN, CRNN 
            #model.save('LSTM_Crying.h5')
            
            new_model = tf.keras.models.load_model('./BPFLSTM_Crying.h5')
            #clf_from_joblib2 = joblib.load('./LSTMmodel.pkl')
            #new_model1 = tf.keras.models.load_model('학부생 Crying detection\CNN_Crying.h5')
            #new_model2 = tf.keras.models.load_model('학부생 Crying detection\RCNN_Crying.h5')
            
            #for i in range(len(X_test11)):

            #X_test111 = X_test11[0][:10000]
            
            
            #X_test111 = X_test111.reshape((1, 10000, 1))    
            new_hat = new_model.predict(np.array(X_test11))
            #KNN
            

            if new_hat[0][0] <= -100:
                    print("LSTM : Crying")
            else:
                    print("LSTM : Noise")
            
           
            break

            #if os.path.isfile(WAVE_OUTPUT_FILENAME):
                #os.remove(WAVE_OUTPUT_FILENAME)
            

            #print( mfcc.shape )


            # stop Recording
        
except KeyboardInterrupt:
    stream.stop_stream()
    stream.close()
    audio.terminate()


C:\Users\82104\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


1590
Crying Detection
recording...
finished recording
LSTM : Crying
